In [1]:
#! pip install dwave-ocean-sdk

In [2]:
import os
import itertools
import click
import pandas as pd
import numpy as np
from dwave.system import LeapHybridCQMSampler, LeapHybridSampler
from dimod import ConstrainedQuadraticModel, BinaryQuadraticModel, QuadraticModel, quicksum, dimod
from dotenv import load_dotenv, find_dotenv

In [3]:
# Load Env Variables
load_dotenv(find_dotenv())
token = os.environ['DWAVE_API_KEY'] 

In [4]:
def parse_inputs(data_file, capacity):
    """Parse user input and files for data to build CQM.

    Args:
        data_file (csv file):
            File of items (weight & cost) slated to ship.
        capacity (int):
            Max weight the shipping container can accept.

    Returns:
        Costs, weights, and capacity.
    """
    df = pd.read_csv(data_file, names=['cost', 'weight'])

    if not capacity:
        capacity = int(0.8 * sum(df['weight']))
        print("\nSetting weight capacity to 80% of total: {}".format(str(capacity)))

    return df['cost'], df['weight'], capacity

In [5]:
def build_knapsack_cqm(costs, weights, max_weight):
    """Construct a CQM for the knapsack problem.

    Args:
        costs (array-like):
            Array of costs for the items.
        weights (array-like):
            Array of weights for the items.
        max_weight (int):
            Maximum allowable weight for the knapsack.

    Returns:
        Constrained quadratic model instance that represents the knapsack problem.
    """
    num_items = len(costs)

    # y_j indicates that item i is used
    y_labels = [f'{i}' for i in range(num_items)]
    print("\nBuilding a CQM for {} items.".format(str(num_items)))

    cqm = ConstrainedQuadraticModel()

    # Set the objective: maximime the profits associated with the value of the items
    objective = QuadraticModel()
    objective.add_linear_from(
                        (
                        (y_labels[i], -costs[i]) for i in range(num_items) 
                        ), 
                        default_vartype='BINARY'
                        )

    cqm.set_objective(objective)

    # Set the constraint related to the knapsack capacity
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (y_labels[i], weights[i]) for i in range(num_items) 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="<=", rhs=max_weight, label='capacity')
    return cqm

In [6]:
def parse_solution(sampleset, costs, weights):
    """Translate the best sample returned from solver to shipped items.

    Args:

        sampleset (dimod.Sampleset):
            Samples returned from the solver.
        costs (array-like):
            Array of costs for the items.
        weights (array-like):
            Array of weights for the items.
    """
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

    if not len(feasible_sampleset):
        raise ValueError("No feasible solution found")

    best = feasible_sampleset.first

    selected_item_indices = [key for key, val in best.sample.items() if val==1.0]
    selected_item_indices = [eval(i) for i in selected_item_indices]
    selected_weights = list(weights.loc[selected_item_indices])
    selected_costs = list(costs.loc[selected_item_indices])
    
    print("\nFound best solution at energy {}".format(best.energy))
    print("\nSelected item numbers (0-indexed):", selected_item_indices)
    print("\nSelected item weights: {}, total = {}".format(selected_weights, sum(selected_weights)))
    print("\nSelected item costs: {}, total = {}".format(selected_costs, sum(selected_costs)))

In [7]:
filename = 'https://raw.githubusercontent.com/dwave-examples/knapsack/master/data/small.csv'
capacity = ''

In [8]:
costs, weights, capacity = parse_inputs(filename, capacity)


Setting weight capacity to 80% of total: 89


In [9]:
cqm = build_knapsack_cqm(costs, weights, capacity)


Building a CQM for 7 items.


In [10]:
sampler = LeapHybridCQMSampler(token=token)
print("Submitting CQM to solver {}.".format(sampler.solver.name))
sampleset = sampler.sample_cqm(cqm, label='Example - Knapsack')

Submitting CQM to solver hybrid_constrained_quadratic_model_version1.


In [11]:
parse_solution(sampleset, costs, weights)


Found best solution at energy -340.0

Selected item numbers (0-indexed): [1, 3, 4, 5, 6]

Selected item weights: [27, 17, 20, 10, 15], total = 89

Selected item costs: [85, 50, 70, 80, 55], total = 340
